## 지도학습  

### 분류 - 이진분류 
- 목표변수가 2개의 범주인 것 

In [1]:
import pandas as pd

In [2]:
df_TFD = pd.read_csv('../../../datasets/TitanicFromDisaster_train.csv')
df_TFD[:2]

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C


### 전처리 과정

In [3]:
df_TFD.columns

Index(['PassengerId', 'Survived', 'Pclass', 'Name', 'Sex', 'Age', 'SibSp',
       'Parch', 'Ticket', 'Fare', 'Cabin', 'Embarked'],
      dtype='object')

In [4]:
df_TFD_extract = df_TFD[['Survived', 'Pclass', 'Age']]
df_TFD_extract.isnull().sum()

Survived      0
Pclass        0
Age         177
dtype: int64

In [5]:
df_TFD_extract_preprocess = df_TFD_extract.dropna()
df_TFD_extract_preprocess[:2]

,Survived,Pclass,Age
0,0,3,22.0
1,1,1,38.0


#### Scaling & Encoding 
- split 하기 전에 실시하는게 유리 

##### Encoding with One Hot Encoding 
- 범주형 데이터를 연속형 숫자 데이터로 변환
- 범주형 항목만큼 벡터화

In [6]:
df_TFD_extract_preprocess['Pclass'].value_counts()

3    355
1    186
2    173
Name: Pclass, dtype: int64

In [7]:
from sklearn.preprocessing import OneHotEncoder

In [8]:
oneHotEncoder = OneHotEncoder() #인스턴스화 
oneHotEncoder.fit(df_TFD_extract_preprocess[['Pclass']]) ## value값만 넣어줘야 함 [[]]을 통해 value만 넣어줌
# 해당 항목 학습한 것임.

## .value_counts() 항목만 넣어줘도 됨 / 몇개의 항목이 있는지 모를경우 전체의 값을 집어넣음

OneHotEncoder()

In [9]:
column_name = oneHotEncoder.categories_  # 컬럼 name

In [10]:
# oneHotEncoder.transform(df_TFD_extract_preprocess[['Pclass']]).toarray() #values # 실제 값 확인용 
encoded_data = oneHotEncoder.transform(df_TFD_extract_preprocess[['Pclass']]).toarray()

In [11]:
df_TFD_extract_preprocess[['Pclass']][:3]

,Pclass
0,3
1,1
2,3


In [12]:
## 병합 위해 numpy array to DataFrame
df_encoded_data = pd.DataFrame(data=encoded_data, columns=oneHotEncoder.get_feature_names_out(['Pclass']))
df_encoded_data[:2]

## columns=oneHotEncoder.get_feature_names_out(['Pclass'] 컬럼명을 Pclass로 임의로 

,Pclass_1,Pclass_2,Pclass_3
0,0.0,0.0,1.0
1,1.0,0.0,0.0


In [13]:
df_encoded_data.index, df_encoded_data.shape

## 인덱스의 갯수가 안맞음 

(RangeIndex(start=0, stop=714, step=1), (714, 3))

In [14]:
df_TFD_extract_preprocess.index, df_TFD_extract_preprocess.shape

(Int64Index([  0,   1,   2,   3,   4,   6,   7,   8,   9,  10,
             ...
             880, 881, 882, 883, 884, 885, 886, 887, 889, 890],
            dtype='int64', length=714),
 (714, 3))

#### concat(병합)

In [15]:
df_TFD_extract_preprocess['Pclass'].shape

(714,)

In [16]:
# df_encoded_data = pd.get_dummies(df_TFD_extract_preprocess['Pclass'], prefix='Pclass')  ## prefix='Pclass' 컬럼명을 부여

In [17]:
df_TFD_extract_preprocess = pd.concat([df_TFD_extract_preprocess.reset_index(drop=True), df_encoded_data.reset_index(drop=True)], axis=1)
df_TFD_extract_preprocess[:2]

,Survived,Pclass,Age,Pclass_1,Pclass_2,Pclass_3
0,0,3,22.0,0.0,0.0,1.0
1,1,1,38.0,1.0,0.0,0.0


In [18]:
## 원래 df에 새로 생성한 df를 붙이는 과정 
#pd.concat([df_TFD_extract_preprocess, df_encoded_data], axis=1, ignore_index=True)

#### Scaling - MinMaxScaler
- 목표변수는 해당사항 아님 

In [19]:
df_TFD_extract_preprocess.shape

(714, 6)

In [20]:
target = df_TFD_extract_preprocess['Survived']

In [21]:
features = df_TFD_extract_preprocess.drop(columns=['Survived', 'Pclass'])

In [22]:
features.columns

Index(['Age', 'Pclass_1', 'Pclass_2', 'Pclass_3'], dtype='object')

In [23]:
from sklearn.preprocessing import MinMaxScaler

In [24]:
# 대상이 같을때 fit_transform를 동시에 
minMaxScaler = MinMaxScaler() #인스턴스화
features = minMaxScaler.fit_transform(features)
features.shape
# 가장큰게 1, 작은게 0

(714, 4)

#### 정형화 단계 

In [25]:
from sklearn.model_selection import train_test_split
# target = df_TFD_extract_preprocess['Survived']
# features = df_TFD_extract_preprocess.drop(columns=['Survived'])

In [67]:
features_train, features_test, target_train, target_test = train_test_split(features, target, random_state=111) #random_state=111 랜덤값으로 설정
features_train.shape, target_train.shape, features_test.shape, target_test.shape

## 총 4개의 결과값 / train은 학습, test 시험, 평가
# 데이터가 500개가 넘지 않으면 split을 하지 않고 데이터를 더 모아야함

((535, 4), (535,), (179, 4), (179,))

### 모델 학습

In [69]:
from sklearn.linear_model import LogisticRegression
model = LogisticRegression() ## 인스턴스화 
model.fit(features_train, target_train)

LogisticRegression()

In [70]:
model.coef_, model.intercept_

(array([[-2.59255724,  1.11141847,  0.02997489, -1.14146766]]),
 array([0.86447724]))

### 예측

In [71]:
df_TFD_extract_preprocess[10:15]

,Survived,Pclass,Age,Pclass_1,Pclass_2,Pclass_3
10,1,1,58.0,1.0,0.0,0.0
11,0,3,20.0,0.0,0.0,1.0
12,0,3,39.0,0.0,0.0,1.0
13,0,3,14.0,0.0,0.0,1.0
14,1,2,55.0,0.0,1.0,0.0


In [72]:
model.predict(features_train[10:15])

## 실제값 : 1,0,0,0,1 / 결과값 : [0, 0, 0, 0, 0]

array([0, 0, 0, 1, 0], dtype=int64)

In [73]:
model.predict_proba(features_train[10:15]) ## predict_proba : 확률값

## 0일 확률 : 0.52507531 / 1일 확률 : 0.47492469

array([[0.59715375, 0.40284625],
       [0.63564456, 0.36435544],
       [0.75219168, 0.24780832],
       [0.47500723, 0.52499277],
       [0.73984819, 0.26015181]])

### 평가 
- accuracy_score(정확도)의 값이 0.85보다 높아야 데이터의 의미가 있음
    - 0.85 이하는 모델자체가 성능이 떨어진다는 의미 

In [74]:
target_train_predict = model.predict(features_train) # 설명변수로 목표변수 예측 / rf)잘 교육을 받았는지 시험을 친다
target_train_predict.shape #target_train.shape 동일

(535,)

In [75]:
from sklearn.metrics import accuracy_score # 정확도

In [76]:
accuracy_score(target_train, target_train_predict) ## 실제값과 교육한 값과 비교(교내 시험)

# 0.696078431372549 정규화 이전 

0.708411214953271

In [77]:
target_test_predict = model.predict(features_test) # test한 값의 예측값
target_test_predict.shape #target_test.shape과 동일

(179,)

In [78]:
accuracy_score(target_test, target_test_predict)
## 실제test값과 시험값의 예측의 비교(교외 시험)

# 0.6703910614525139 정규화 이전 

0.6480446927374302

#### 추가평가 도구 
- MSE (mean_squared_error, 에러율)
    - 회귀 모델의 성능을 평가하는데 사용되며, 예측 값과 실제 값 사이의 오차를 평가하는 도구
    - 작을수록 모델의 예측이 실제 값과 가깝다고 판단, MSE가 작으면 모델이 데이터를 잘 설명하고 있음을 나타내며, 더 나은 회귀 모델일 가능성이 높음

In [79]:
from sklearn.metrics import mean_squared_error, mean_absolute_error

In [80]:
mean_squared_error(target_train, target_train_predict) # train에 대한 평가

0.29158878504672897

In [81]:
## root mean_squared_error(RMSE)
mean_squared_error(target_train, target_train_predict)**0.5

0.539989615684162

In [82]:
##mean_absolute_error(MAE)
mean_absolute_error(target_train, target_train_predict) # train에 대한 평가

0.29158878504672897

#### F1
- shape에서 갯수가 같아야 report를 쓸수 있음 
- precision : 정밀도 
- recall : 재현율

In [83]:
from sklearn.metrics import classification_report

In [84]:
print(classification_report(target_train, target_train_predict)) #교육

## support ; record의 갯수 
## recall ; 한쪽으로 값이 치우쳐져 있음 --> 죽은 데이터가 많다.

              precision    recall  f1-score   support

           0       0.72      0.82      0.77       312
           1       0.69      0.55      0.61       223

    accuracy                           0.71       535
   macro avg       0.70      0.69      0.69       535
weighted avg       0.71      0.71      0.70       535



In [85]:
print(classification_report(target_test, target_test_predict)) #시험

              precision    recall  f1-score   support

           0       0.72      0.71      0.71       112
           1       0.53      0.55      0.54        67

    accuracy                           0.65       179
   macro avg       0.63      0.63      0.63       179
weighted avg       0.65      0.65      0.65       179



### 오차 행렬 (confusion_matrix)
- precision : 정밀도 
- recall : 재현율

In [86]:
from sklearn.metrics import confusion_matrix, precision_score, recall_score, f1_score

In [87]:
confusion_matrix(target_train, target_train_predict)

array([[257,  55],
       [101, 122]], dtype=int64)

In [88]:
precision_score(target_train, target_train_predict)

0.6892655367231638

In [89]:
recall_score(target_train, target_train_predict)

0.547085201793722

In [90]:
f1_score(target_train, target_train_predict)

0.6100000000000001

### 서비스 

In [91]:
df_TFD_extract[100:103]

,Survived,Pclass,Age
100,0,3,28.0
101,0,3,NaN
102,0,1,21.0


In [92]:
# 예측 대상자 입력값 Pclass: 3, age 28.0 --> 임의의 값
encoder_pclass_= oneHotEncoder.transform([[3]]).toarray() #2차원으로 넣어줘야함 
encoder_pclass_, encoder_pclass_.flatten()

#.flatten() --> 차원 배열(또는 행렬)을 1차원 배열로 변환하는 함수

C:\Users\02-19\anaconda3\Lib\site-packages\sklearn\base.py:439: UserWarning: X does not have valid feature names, but OneHotEncoder was fitted with feature names
  warnings.warn(


(array([[0., 0., 1.]]), array([0., 0., 1.]))

In [93]:
import numpy as np 
inputs_data = np.concatenate(([28.0],encoder_pclass_.flatten()))
inputs_data

## 임의의 값을 모델에 넣어주는 값의 형식과 동일하게 됨

array([28.,  0.,  0.,  1.])

In [94]:
model.predict([inputs_data]) #목표변수 얻기 

array([0], dtype=int64)